In [9]:
%pip install supabase


[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import supabase
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import SupabaseVectorStore
from langchain.document_loaders import TextLoader

from supabase.client import Client, create_client

import json

In [11]:
SUPABASE_URL = os.environ['NEXT_PUBLIC_SUPABASE_URL']
SUPABASE_KEY = os.environ['NEXT_PUBLIC_SUPABASE_ANON_KEY']
supabase_client = supabase.create_client(SUPABASE_URL, SUPABASE_KEY)

In [12]:
for i in range(2):

    filename = f'/Users/vdutts7/Downloads/ee16b/config/transcripts/lecture{i+1}.txt'
    with open(filename) as f:
        lecture_file = f.read()
    
    rec_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 900,
        chunk_overlap  = 100,
        length_function = len,
    )

    texts = rec_text_splitter.create_documents([lecture_file])

    jsonfilename = f'/Users/vdutts7/Downloads/ee16b/config/jsons/lecture{i+1}.json'
    with open(jsonfilename) as f2:
        lecture_metadata = json.load(f2)
    for text in texts:
        text.metadata = lecture_metadata


    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    
    vector_store = SupabaseVectorStore.from_documents(
        texts, 
        embeddings, 
        client=supabase_client
    )

2023-06-04 11:11:43,996:WARNING - Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).
